In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Conv2D, Conv2DTranspose, PReLU, BatchNormalization, Add
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam # Optimizer
from tensorflow.keras.applications.vgg19 import VGG19 # Perceptual loss
from tensorflow.keras.losses import MeanSquaredError # Loss function
from tensorflow.image import psnr, ssim # Evaluation metrics
from tensorflow.keras.utils import Progbar

2023-04-11 19:18:08.973377: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
# Image dimension (height, width, color)
lr_shape = (255, 255, 3)
hr_shape = (1020, 1020, 3)

In [3]:
# Build VGG19 model to use as perceptual loss (10 layers)
def build_vgg():
    
    vgg = VGG19(weights='imagenet', include_top=False, input_shape=hr_shape)
    
    return Model(inputs=vgg.inputs, outputs=vgg.layers[10].output)

In [4]:
vgg = build_vgg()
vgg.summary()

2023-04-11 19:18:14.385875: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2023-04-11 19:18:14.387297: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2023-04-11 19:18:14.462580: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:12:00.0 name: Tesla V100-PCIE-32GB computeCapability: 7.0
coreClock: 1.38GHz coreCount: 80 deviceMemorySize: 31.75GiB deviceMemoryBandwidth: 836.37GiB/s
2023-04-11 19:18:14.463126: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 1 with properties: 
pciBusID: 0000:13:00.0 name: Tesla V100-PCIE-32GB computeCapability: 7.0
coreClock: 1.38GHz coreCount: 80 deviceMemorySize: 31.75GiB deviceMemoryBandwidth: 836.37GiB/s
2023-04-11 19:18:14.463661: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 2 with properties: 
pciBusID: 0000:14:00.0 name: Tesl

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1020, 1020, 3)]   0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 1020, 1020, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 1020, 1020, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 510, 510, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 510, 510, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 510, 510, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 255, 255, 128)     0     

In [5]:
# Residual block
def residual_block(x):

    input_tensor = x
    
    x = Conv2D(64, kernel_size=3, padding = "same")(x)
    x = BatchNormalization()(x)
    x = PReLU(shared_axes = [1,2])(x)

    x = Conv2D(64, kernel_size=3, padding = "same")(x)
    x = BatchNormalization()(x)

    return Add()([input_tensor, x])

In [6]:
# Upscale the image 2x
def upscale_block(x):   
    
    x = Conv2DTranspose(256, kernel_size=3, strides=2, padding="same")(x)
    x = PReLU(shared_axes=[1,2])(x)

    return x

In [7]:
num_residual_block = 16

In [8]:
# resnet model
def build_resnet():
    
    lr_input = Input(shape=lr_shape)
    
    x = Conv2D(64, kernel_size=9, padding="same")(lr_input)
    x = PReLU(shared_axes=[1,2])(x)
    temp = x
    
    for i in range(num_residual_block):
        x = residual_block(x)
    
    x = Conv2D(64, kernel_size=3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Add()([x, temp])
    
    x = upscale_block(x)
    x = upscale_block(x)
    
    hr_image = Conv2D(3, kernel_size=9, padding="same")(x)
    
    return Model(inputs=lr_input, outputs=hr_image)

In [9]:
resnet = build_resnet()
resnet.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 255, 255, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 255, 255, 64) 15616       input_2[0][0]                    
__________________________________________________________________________________________________
p_re_lu (PReLU)                 (None, 255, 255, 64) 64          conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 255, 255, 64) 36928       p_re_lu[0][0]                    
____________________________________________________________________________________________

In [10]:
# Add VGG19 to resnet model
def resnet_vgg(resnet, vgg):
    
    vgg.trainable = False

    # Input layer
    lr_input = Input(shape=lr_shape)
    
    hr_image = resnet(lr_input)
    hr_features = vgg(hr_image)
    
    model = Model(inputs=lr_input, outputs=hr_features)
    
    return model

In [11]:
resnet_vgg = resnet_vgg(resnet, vgg)
resnet_vgg.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 255, 255, 3)]     0         
_________________________________________________________________
model_1 (Functional)         (None, 1020, 1020, 3)     2044291   
_________________________________________________________________
model (Functional)           (None, 255, 255, 256)     2325568   
Total params: 4,369,859
Trainable params: 2,040,067
Non-trainable params: 2,329,792
_________________________________________________________________


In [12]:
# Determine evaluation metrics

# Determine color max range
max_val = 1.0

# Peak signal-to-noise ratio (PSNR)
def PSNR(y_true, y_pred):
    return psnr(y_true, y_pred, max_val=max_val)

# Structural similarity index measure (SSIM)
def SSIM(y_true, y_pred):
    return ssim(y_true, y_pred, max_val=max_val)

In [13]:
# Load the low-resolution images for training
lr_images_train = np.load('../Datasets/lr_images_un_train.npy')/255.0
# lr_images_train = np.load('lr_images_un_train.npy')[0:10]/255.0

# Load the high-resolution images for training
hr_images_train = np.load('../Datasets/hr_images_train.npy')/255.0
# hr_images_train = np.load('hr_images_train.npy')[0:10]/255.0

# Load the low-resolution images for validation
lr_images_val = np.load('../Datasets/lr_images_un_val.npy')/255.0
# lr_images_val = np.load('lr_images_un_val.npy')[0:50]/255.0

# Load the high-resolution images for validation
hr_images_val = np.load('../Datasets/hr_images_val.npy')/255.0
# hr_images_val = np.load('hr_images_val.npy')[0:50]/255.0

In [14]:
# n-th tries
model_name = 'resnet_un'
n = 3

batch_size = 1
epochs = 25

In [15]:
val_features_1 = vgg.predict(hr_images_val[:50], batch_size=batch_size)
val_features_2 = vgg.predict(hr_images_val[50:], batch_size=batch_size)
hr_features_val = np.append(val_features_1, val_features_2, axis=0)

2023-04-11 19:18:38.949956: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2023-04-11 19:18:38.962296: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2500000000 Hz
2023-04-11 19:18:39.135639: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7
2023-04-11 19:18:40.748076: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10


In [16]:
resnet_vgg.compile(optimizer=Adam(), loss=MeanSquaredError(), metrics=['accuracy'])

In [17]:
# Train the models
for epoch in range(epochs):

    print('Epoch %d/%d' % (epoch+1, epochs))
    progbar = Progbar(lr_images_train.shape[0] // batch_size)

    for i in range(lr_images_train.shape[0] // batch_size):
        # Randomly sample a batch of images
        idx = np.random.randint(0, lr_images_train.shape[0], batch_size)
        lr_batch = lr_images_train[idx]
        hr_batch = hr_images_train[idx]

        # Train resnet
        hr_batch_features = vgg.predict(hr_batch)
        resnet_loss, resnet_acc = resnet_vgg.train_on_batch(lr_batch, hr_batch_features)

        # Update progress bar
        progbar.update(i+1, [('Model Loss', resnet_loss), ('Accuracy', resnet_acc)])

    # Evaluate the generator model on the validation dataset
    # print("Validating generator.")
    resnet_loss_val, resnet_acc_val  = resnet_vgg.evaluate(lr_images_val, hr_features_val, batch_size=batch_size)
    
    # Save the generator model every epochs
    resnet.save('../model/%s%d_%02dof%d.h5' % (model_name, n, epoch+1, epochs))

Epoch 1/25
100/100 [==============================] - 21s 197ms/step - loss: 31.9528 - accuracy: 0.7858
Epoch 2/25
100/100 [==============================] - 20s 198ms/step - loss: 21.5092 - accuracy: 0.8131
Epoch 3/25
100/100 [==============================] - 20s 198ms/step - loss: 21.9104 - accuracy: 0.8106
Epoch 4/25
100/100 [==============================] - 20s 195ms/step - loss: 24.6217 - accuracy: 0.7945
Epoch 5/25
100/100 [==============================] - 20s 192ms/step - loss: 18.3762 - accuracy: 0.8291
Epoch 6/25
100/100 [==============================] - 20s 195ms/step - loss: 17.1740 - accuracy: 0.8302
Epoch 7/25
100/100 [==============================] - 19s 191ms/step - loss: 16.7765 - accuracy: 0.8322
Epoch 8/25
100/100 [==============================] - 20s 195ms/step - loss: 14.4448 - accuracy: 0.8442
Epoch 9/25
100/100 [==============================] - 20s 192ms/step - loss: 15.5840 - accuracy: 0.8416
Epoch 10/25
100/100 [==============================] - 20s 192ms